In [ ]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, functions as F
import re
import numpy as np

In [2]:
startpath ="_Star_Ratings_and_Display_Measures/"
fallpath ="_Star_Ratings_Fall_Release/"
springpath ="_Star_Ratings_Spring_Release/"
cpath="_Part_C"
dpath="_Part_D"
finalpath ="_Report_Card_Master_Table.xlsx"
year = "2014"



In [3]:
firstfive = ["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization"]

In [35]:
y = 2017
path = f"./Data/{y}{startpath}{y}{fallpath}{y}{finalpath}"
dfms = pd.read_excel(path,"Measure_Stars",header=2)
dfms = dfms.iloc[1:]
for x in range(5,len(dfms.keys())):
    dfms =dfms.rename(columns= {dfms.keys()[x] : re.split(r'\d+:',dfms.keys()[x])[0]+re.split(r'\d+:',dfms.keys()[x])[1]})
for x in range(5):
    dfms = dfms.rename(columns={f"Unnamed: {x}":firstfive[x]})
dfms

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Annual Flu Vaccine,C Improving or Maintaining Physical Health,C Improving or Maintaining Mental Health,...,D Beneficiary Access and Performance Problems,D Drug Plan Quality Improvement,D Rating of Drug Plan,D Getting Needed Prescription Drugs,D MPF Price Accuracy,D High Risk Medication,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR
1,E0654,Employer/Union Only Direct Contract PDP,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Pro...,IBT Voluntary Employee Benefits Trust,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,5,4,5,Not enough data available,4,3,4,3,3
2,E2630,Employer/Union Only Direct Contract PDP,C AND O EMPLOYEES' HOSPITAL ASSOCIATION,C and O Employees Hospital Association,C & O Employees' Hospital Association,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,4,3,5,5,Not enough data available,3,3,4,4,2
3,E3014,Employer/Union Only Direct Contract PDP,PSERS HOP PROGRAM,Pennsylvania Public School Employees Retiremen...,Commonwealth of PA Pub Schools Retirement System,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,5,5,Not enough data available,5,4,5,4,3
4,E4744,Employer/Union Only Direct Contract PDP,MODOT/MSHP MEDICAL AND LIFE INSURANCE PLAN,MISSOURI DEPARTMENT OF TRANSPORTATION,Missouri Highways and Transportation Commission,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,4,5,Not enough data available,4,4,5,4,3
5,E7316,Employer/Union Only Direct Contract PDP,UNION PACIFIC RAILROAD EMPLOYES HEALTH SYSTEMS,Union Pacific Railroad Employes Health Systems,Union Pacific Railroad Employes Health Systems,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,2,5,2,Not enough data available,3,1,1,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,S7694,PDP,ENVISION INSURANCE COMPANY,EnvisionRx Plus,Rite Aid Corporation,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,3,4,3,3,5,5,2,1,2,3
604,S8067,PDP,AVALON INSURANCE COMPANY,Avalon Insurance Company,Capital BlueCross,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,1,4,3,3,5,5,5,5,4,4
605,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,2,2,Not enough data available,2,3,4,3,5
606,S9579,PDP,TRANSAMERICA LIFE INSURANCE COMPANY,Transamerica Life Insurance Company,Aegon N.V.,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,3,5,5,5,3,3,3,3,3


In [36]:
def get_measure_stars(path):
    dfms = pd.read_excel(path,"Measure_Stars",header=2)
    dfms = dfms.iloc[1:]
    for x in range(5,len(dfms.keys())):
        dfms =dfms.rename(columns= {dfms.keys()[x] : re.split(r'\d+:',dfms.keys()[x])[0]+re.split(r'\d+:',dfms.keys()[x])[1]})
    for x in range(5):
        dfms = dfms.rename(columns={f"Unnamed: {x}":firstfive[x]})
    return dfms

In [5]:
def get_domain_stars(path):
    dfds = pd.read_excel(path,"Domain_Stars",header=1)
    return dfds

In [6]:
def get_summary_rating(path):
    dfsr = pd.read_excel(path,"Summary_Rating",header=1)
    dfsr = dfsr.drop(columns="Sanction Deduction",errors='ignore')
    return dfsr

In [7]:
def get_early(y):
    fullpath = f"./Data/{y}{startpath}{y}{fallpath}{y}{finalpath}"
    dfmsf = get_measure_stars(fullpath)
    dfdsf = get_domain_stars(fullpath)
    dfsrf = get_summary_rating(fullpath)
    dff = pd.merge(dfmsf,dfdsf,on=firstfive,how='left')
    dff = pd.merge(dff,dfsrf,on=firstfive,how='left')
    dff["Year"] = f"{y} Fall"
    dff = dff.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    fullpath = f"./Data/{y}{startpath}{y}{springpath}{y}{finalpath}"
    dfmss = get_measure_stars(fullpath)
    dfdss = get_domain_stars(fullpath)
    dfsrs = get_summary_rating(fullpath)
    dfs = pd.merge(dfmss,dfdss,on=firstfive,how='left')
    dfs = pd.merge(dfs,dfsrs,on=firstfive,how='left')
    dfs["Year"] = f"{y} Spring"
    dfs = dfs.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    df = pd.concat([dff,dfs])
    df = df.drop(columns = "2017 Disaster %",errors='ignore')
    return df

In [8]:
def get_late(y):
    fullpath = f"./Data/{y}{startpath}{y}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    #drop disaster columns
    dfsrc = dfsrc.drop(columns=dfsrc.columns[[6,7]],axis = 1)
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    df = df.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    df["Year"] = f"{y}"
    return    df

In [44]:
def standardize_data():
    year = 2014
    years = [x for x in range(2015,2026)]        
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{cpath}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{dpath}{finalpath}"
    dfmsd = get_measure_stars(fullpath)
    dfdsd = get_domain_stars(fullpath)
    dfsrd = get_summary_rating(fullpath)
    df = pd.merge(df,dfmsd,on=firstfive,how='left')
    df = pd.merge(df,dfdsd,on=firstfive,how='left')
    df = pd.merge(df,dfsrd,on=firstfive,how='left')
    df = df.rename(columns={f"{year} Part C Summary Rating":"Part C Summary",f"{year} Overall Rating":"Overall",f"{year} Part D Summary Rating":"Part D Summary"})
    fullpath = f"./Data/{year}{startpath}{year}{springpath}{year}{cpath}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    df1 = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df1 = pd.merge(df1,dfsrc,on=firstfive,how='left')
    fullpath = f"./Data/{year}{startpath}{year}{springpath}{year}{dpath}{finalpath}"
    dfmsd = get_measure_stars(fullpath)
    dfdsd = get_domain_stars(fullpath)
    dfsrd = get_summary_rating(fullpath)
    df1 = pd.merge(df1,dfmsd,on=firstfive,how='left')
    df1 = pd.merge(df1,dfdsd,on=firstfive,how='left')
    df1 = pd.merge(df1,dfsrd,on=firstfive,how='left')    
    df1 = df1.rename(columns={f"{year} Part C Summary Rating":"Part C Summary",f"{year} Overall Rating":"Overall","{year} Part D Summary Rating":"Part D Summary"})    
    df["Year"] = "2014 Fall"
    df1["Year"] = "2014 Spring"
    finaldf = pd.concat([df,df1])
    #spark = SparkSession.builder.appName("pandas to spark").getOrCreate()
    #sparkdf = spark.createDataFrame(finaldf)
    #sparkdf.show()
    for year in years:
        if (year <2020):
            df = get_early(year)
        else:
            df = get_late(year)
        #placeholderdf = spark.createDataFrame(df)
        #placeholderdf.show()
        newdf = pd.concat([finaldf,df],axis=0,join='outer')
        finaldf=newdf
    finaldf = finaldf.reset_index(drop=True)
    return finaldf

In [78]:
def clean_data(combodf1):
    for x in range(len(combodf1.keys())):
        combodf1[combodf1.keys()[x]] = combodf1[combodf1.keys()[x]].astype(str)
    combodf1 = combodf1.replace('\\s*Plan\\s*too\\s*new\\s*to\\s*be\\s*measured\\s*','Plan too new to be measured',regex=True)
    combodf1 = combodf1.replace('\\s*Plan\\s*too\\s*small\\s*to\\s*be\\s*measured\\s*','Plan too small to be measured',regex=True)
    combodf1 = combodf1.replace('\\s*Plan\\s*not\\s*required\\s*to\\s*report\\s*measure\\s*','Plan not required to report measure',regex=True)
    combodf1 = combodf1.replace('\\s*No\\s*data\\s*available\\s*','No data available',regex=True)
    combodf1 = combodf1.replace('\\s*Not\\s*enough\\s*data\\s*available\\s*','Not enough data available',regex=True)
    return combodf1

In [ ]:
def cull_data(df):
    

In [ ]:
combodf = standardize_data()
combodf.to_csv("./Data/CombinedData.csv",header=True,index=False)

In [77]:
combodf = clean_data(combodf)
combodf.to_csv("./Data/CombinedDataCleaned.csv",header=True,index=False)

In [79]:
combodf.loc[combodf['C Breast Cancer Screening']=='Plan too small to be measured']

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
8,H0141,Local CCP,"MCLAREN HEALTH PLAN, INC.",McLarenAdvantage,McLaren Health Care Corporation,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
18,H0294,Local CCP,CARE IMPROVEMENT PLUS WISCONSIN INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
31,H0408,Local CCP,UNITEDHEALTHCARE INSURANCE COMPANY,UnitedHealthcare,"UnitedHealth Group, Inc.",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
36,H0490,Local CCP,"MOLINA HEALTHCARE OF OHIO, INC.",Molina Healthcare of Ohio,"Molina Healthcare, Inc.,",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
37,H0502,1876 Cost,THE CONTRA COSTA HEALTH PLAN,Contra Costa Health Plan,Contra Costa Health Services,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12645,H9589,Local CCP,"CLEAR SPRING HEALTH (GA), INC.",Clear Spring Health,Group 1001,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,Not enough data available,Not enough data available,Not enough data available,Not enough data available,Plan too small to be measured,Plan too small to be measured,Not enough data available,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12662,H9826,Local CCP,"COMMUNITY HEALTH CHOICE TEXAS, INC.",Community Health Choice,Harris County Hospital District,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,2,2,1,2,Plan too small to be measured,Plan too small to be measured,4,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12663,H9827,Local CCP,"MOUNT CARMEL HEALTH PLAN OF NEW YORK, INC.",Trinity Health Plan New York,Trinity Health Corporation,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,4,5,4,Not enough data available,Plan too small to be measured,Plan too small to be measured,5,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12667,H9888,Local CCP,DEVOTED HEALTH INSURANCE COMPANY OF ALABAMA INC,Devoted Health,"Devoted Health, Inc.",Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,1,2,2,Not enough data available,Plan too small to be measured,Plan too small to be measured,3,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured


In [ ]:
combodf2 = combodf.copy()
combodf2 = combodf2.fillna(0)
combodf2 = combodf2.replace('Plan too new to be measured',np.nan)
combodf2 = combodf2.dropna(how='any')
combodf2 = combodf2.replace('Not enough data available',0)
